# Semantic Kernel エージェントフレームワーク比較

このコードサンプルでは、[Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AIフレームワークを使用した高度なエージェント実装を探求します。

## 📚 このノートブックで学習できること

### 🎯 主要な学習目標
- **Semantic Kernel**の高度な機能活用
- **プラグインシステム**による機能拡張
- **他フレームワークとの比較**理解
- **エンタープライズ対応**の実装パターン

### 🛠️ 実装する技術スタック
- **Semantic Kernel**: Microsoft製AIエージェントフレームワーク
- **プラグイン開発**: カスタム機能の統合
- **メモリ管理**: 会話履歴の永続化
- **エラーハンドリング**: 本格的な例外処理

### 💡 フレームワーク比較のポイント
- **Semantic Kernel vs AutoGen**: 設計思想の違い
- **単一エージェント vs マルチエージェント**: 用途による使い分け
- **企業導入**: スケーラビリティと管理性

## 必要なパッケージのインストール

In [1]:
%pip install semantic-kernel openai python-dotenv --quiet --upgrade

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## 必要なPythonパッケージのインポート

In [1]:
import os
import asyncio
import json
from typing import Annotated, Any, Dict, List
from datetime import datetime, timedelta

from openai import AsyncOpenAI
from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.kernel import Kernel

## 高度なプラグインシステムの実装

Semantic Kernelの強力なプラグイン機能を活用した、より実用的なプラグインを作成します。

In [2]:
class TravelPlanningPlugin:
    """包括的な旅行計画支援プラグイン"""
    
    def __init__(self):
        # サンプルデータ
        self.destinations = {
            "東京": {
                "spots": ["浅草寺", "東京スカイツリー", "明治神宮", "上野公園"],
                "cuisine": ["寿司", "ラーメン", "天ぷら", "もんじゃ焼き"],
                "budget_per_day": {"economy": 8000, "standard": 15000, "luxury": 30000}
            },
            "京都": {
                "spots": ["清水寺", "金閣寺", "伏見稲荷大社", "嵐山"],
                "cuisine": ["懐石料理", "湯豆腐", "抹茶スイーツ", "京野菜"],
                "budget_per_day": {"economy": 7000, "standard": 12000, "luxury": 25000}
            },
            "大阪": {
                "spots": ["大阪城", "道頓堀", "通天閣", "海遊館"],
                "cuisine": ["たこ焼き", "お好み焼き", "串カツ", "ふぐ料理"],
                "budget_per_day": {"economy": 6000, "standard": 10000, "luxury": 20000}
            }
        }
        
        self.weather_data = {
            "東京": {"temp": "25°C", "condition": "晴れ", "humidity": "60%"},
            "京都": {"temp": "23°C", "condition": "曇り", "humidity": "65%"},
            "大阪": {"temp": "26°C", "condition": "晴れ時々曇り", "humidity": "58%"}
        }

    @kernel_function(description="指定された都市の観光スポット情報を取得")
    def get_tourist_spots(
        self, 
        city: Annotated[str, "観光情報を取得する都市名"]
    ) -> Annotated[str, "観光スポットのリスト"]:
        if city in self.destinations:
            spots = self.destinations[city]["spots"]
            return f"{city}の主要観光スポット: {', '.join(spots)}"
        return f"{city}の観光情報は現在利用できません。"

    @kernel_function(description="指定された都市の現地グルメ情報を取得")
    def get_local_cuisine(
        self, 
        city: Annotated[str, "グルメ情報を取得する都市名"]
    ) -> Annotated[str, "現地グルメのリスト"]:
        if city in self.destinations:
            cuisine = self.destinations[city]["cuisine"]
            return f"{city}の名物グルメ: {', '.join(cuisine)}"
        return f"{city}のグルメ情報は現在利用できません。"

    @kernel_function(description="旅行予算の見積もりを計算")
    def calculate_budget(
        self,
        city: Annotated[str, "旅行先の都市名"],
        days: Annotated[int, "滞在日数"],
        budget_level: Annotated[str, "予算レベル (economy/standard/luxury)"] = "standard"
    ) -> Annotated[str, "予算見積もり"]:
        if city in self.destinations and budget_level in self.destinations[city]["budget_per_day"]:
            daily_budget = self.destinations[city]["budget_per_day"][budget_level]
            total_budget = daily_budget * days
            
            return f"{city}での{days}日間の予算見積もり ({budget_level}レベル): {total_budget:,}円 (1日あたり{daily_budget:,}円)"
        return "予算計算に必要な情報が不足しています。"

    @kernel_function(description="現在の天気情報を取得")
    def get_weather_info(
        self, 
        city: Annotated[str, "天気情報を取得する都市名"]
    ) -> Annotated[str, "天気情報"]:
        if city in self.weather_data:
            weather = self.weather_data[city]
            return f"{city}の現在の天気: {weather['condition']}, 気温: {weather['temp']}, 湿度: {weather['humidity']}"
        return f"{city}の天気情報は現在利用できません。"

class MemoryPlugin:
    """会話履歴とユーザー設定を管理するプラグイン"""
    
    def __init__(self):
        self.user_preferences = {}
        self.conversation_history = []

    @kernel_function(description="ユーザーの旅行設定を保存")
    def save_user_preference(
        self,
        preference_type: Annotated[str, "設定の種類 (budget/style/interest)"],
        value: Annotated[str, "設定値"]
    ) -> Annotated[str, "保存結果"]:
        self.user_preferences[preference_type] = value
        return f"設定を保存しました: {preference_type} = {value}"

    @kernel_function(description="保存されたユーザー設定を取得")
    def get_user_preferences(self) -> Annotated[str, "ユーザー設定"]:
        if self.user_preferences:
            prefs = [f"{k}: {v}" for k, v in self.user_preferences.items()]
            return f"保存された設定: {', '.join(prefs)}"
        return "保存された設定はありません。"

## エージェントの設定とクライアント作成

In [3]:
# 環境変数の読み込み
load_dotenv()

# OpenAIクライアントの作成（GitHub Models使用）
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# チャット完了サービスの設定
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

# プラグインのインスタンス化
travel_plugin = TravelPlanningPlugin()
memory_plugin = MemoryPlugin()

# 高度な旅行エージェントの作成
advanced_travel_agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[travel_plugin, memory_plugin],
    name="AdvancedTravelAgent",
    instructions="""
    あなたは高度な旅行計画支援AIエージェントです。以下の機能を活用してユーザーに最適な旅行プランを提供してください：

    1. **包括的な情報提供**: 観光スポット、グルメ、天気、予算を総合的に考慮
    2. **個人化された提案**: ユーザーの設定や過去の会話を活用
    3. **実用的なアドバイス**: 具体的で実行可能な提案
    4. **予算管理**: 要求に応じて詳細な費用見積もりを提供
    5. **リアルタイム情報**: 天気などの最新情報を含める

    常にユーザーフレンドリーで、質問があれば気軽に聞けるような雰囲気を作ってください。
    """,
)

## エラーハンドリングと会話管理

本格的なアプリケーションを想定したエラーハンドリングと会話管理機能を実装します。

In [4]:
class ConversationManager:
    """会話管理とエラーハンドリングを担当するクラス"""
    
    def __init__(self, agent: ChatCompletionAgent):
        self.agent = agent
        self.active_threads = {}
        self.conversation_count = 0

    async def start_conversation(self, user_id: str = "default") -> str:
        """新しい会話セッションを開始"""
        conversation_id = f"conv_{self.conversation_count}_{user_id}"
        self.conversation_count += 1
        
        try:
            thread = ChatHistoryAgentThread()
            self.active_threads[conversation_id] = thread
            print(f"🟢 会話セッション開始: {conversation_id}")
            return conversation_id
        except Exception as e:
            print(f"❌ 会話開始エラー: {str(e)}")
            raise

    async def send_message(self, conversation_id: str, message: str) -> None:
        """メッセージを送信し、ストリーミング応答を表示"""
        if conversation_id not in self.active_threads:
            raise ValueError(f"会話セッション '{conversation_id}' が見つかりません")
        
        thread = self.active_threads[conversation_id]
        
        try:
            print(f"\n👤 ユーザー: {message}")
            print(f"🤖 {self.agent.name}: ", end="", flush=True)
            
            first_chunk = True
            async for response in self.agent.invoke_stream(
                messages=message, 
                thread=thread
            ):
                if first_chunk:
                    first_chunk = False
                print(response, end="", flush=True)
                
            print("\n")  # 改行
            
        except Exception as e:
            print(f"\n❌ メッセージ送信エラー: {str(e)}")
            # エラー時も会話を継続できるようにする
            await self._send_error_message(thread, str(e))

    async def _send_error_message(self, thread, error_msg: str):
        """エラー時のフォールバック応答"""
        fallback_message = f"申し訳ありません。技術的な問題が発生しました: {error_msg}\n別の質問でお試しください。"
        print(f"🤖 {self.agent.name}: {fallback_message}")

    async def end_conversation(self, conversation_id: str) -> None:
        """会話セッションを終了"""
        if conversation_id in self.active_threads:
            thread = self.active_threads[conversation_id]
            try:
                await thread.delete()
                del self.active_threads[conversation_id]
                print(f"🔴 会話セッション終了: {conversation_id}")
            except Exception as e:
                print(f"⚠️ 会話終了時の警告: {str(e)}")

    async def cleanup_all(self):
        """すべての会話セッションをクリーンアップ"""
        for conv_id in list(self.active_threads.keys()):
            await self.end_conversation(conv_id)

## 実際の会話デモンストレーション

In [5]:
async def demonstration():
    """Semantic Kernelの機能デモンストレーション"""
    
    # 会話マネージャーの初期化
    conversation_mgr = ConversationManager(advanced_travel_agent)
    
    try:
        # 会話セッション開始
        conv_id = await conversation_mgr.start_conversation("demo_user")
        
        # デモンストレーション用の会話シーケンス
        demo_messages = [
            "こんにちは！来月、友人と2泊3日で関西旅行を計画しています。",
            "予算は一人15,000円/日程度で考えています。この設定を保存してください。",
            "京都と大阪、どちらがおすすめですか？観光スポットと天気も教えてください。",
            "京都に決めました！予算見積もりと現地グルメ情報をお願いします。",
            "保存された私の設定を確認してください。"
        ]
        
        print("=" * 60)
        print("🎭 Semantic Kernel 高度なエージェントデモ")
        print("=" * 60)
        
        for message in demo_messages:
            await conversation_mgr.send_message(conv_id, message)
            await asyncio.sleep(1)  # デモ用の間隔
        
        print("\n" + "=" * 60)
        print("✅ デモンストレーション完了")
        print("=" * 60)
        
    except Exception as e:
        print(f"❌ デモ実行エラー: {str(e)}")
    finally:
        # クリーンアップ
        await conversation_mgr.cleanup_all()

# デモンストレーション実行
await demonstration()

🟢 会話セッション開始: conv_0_demo_user
🎭 Semantic Kernel 高度なエージェントデモ

👤 ユーザー: こんにちは！来月、友人と2泊3日で関西旅行を計画しています。
🤖 AdvancedTravelAgent: こんにちは！関西旅行、こんにちは！関西旅行、楽しそうですね楽しそうですね！具体的に！具体的に訪訪れたい都市や観光れたい都市や観光スポット、スポット、食べ食べたいもの、予算たいもの、予算などなどの希望はありますの希望はありますか？それに基づか？それに基づいて旅行プランを提いて旅行プランを提案させていただきます案させていただきます。。




👤 ユーザー: 予算は一人15,000円/日程度で考えています。この設定を保存してください。
🤖 AdvancedTravelAgent: 
👤 ユーザー: 予算は一人15,000円/日程度で考えています。この設定を保存してください。
🤖 AdvancedTravelAgent: 予算の設定を予算の設定を保存しました保存しました！1人！1人ああたり15,000円/日たり15,000円/日ですね。

次に、ですね。

次に、旅行先の都市を決旅行先の都市を決めめましょう。ましょう。例えば、大阪例えば、大阪、京都、、京都、神神戸などが人気戸などが人気ですです。ど。どの都市の都市を訪れたいですか？また、特を訪れたいですか？また、特に興味のあるに興味のある観光スポットやグ観光スポットやグルメがあれば教えてください。

ルメがあれば教えてください。


👤 ユーザー: 京都と大阪、どちらがおすすめですか？観光スポットと天気も教えてください。
🤖 AdvancedTravelAgent: 
👤 ユーザー: 京都と大阪、どちらがおすすめですか？観光スポットと天気も教えてください。
🤖 AdvancedTravelAgent: 京都と大阪の観光スポット京都と大阪の観光スポットと天気の情報をお伝と天気の情報をお伝えしますね。

### 京都のえしますね。

### 京都の観光スポット
観光スポット
- **清水寺- **清水寺**: 有名な**: 有名な木造木造のの寺院で、美しい景色が寺院で、美しい景色が楽しめ楽しめます。
- **金閣ます。
- **金閣寺寺**: 美しい金色の**: 美しい金色の建物が特徴

## 🔄 フレームワーク比較分析

### Semantic Kernel vs AutoGen

| 特徴 | Semantic Kernel | AutoGen |
|------|----------------|---------|
| **設計思想** | 単一エージェント + プラグイン | マルチエージェント会話 |
| **得意分野** | エンタープライズ統合 | 協調的問題解決 |
| **学習コスト** | 中程度 | 高 |
| **スケーラビリティ** | 高（垂直方向） | 高（水平方向） |
| **Microsoft統合** | 深い | 標準的 |

### 🎯 用途別推奨

**Semantic Kernelを選ぶべき場合:**
- エンタープライズアプリケーション統合
- 既存システムへのAI機能追加
- プラグイン中心のアーキテクチャ
- Microsoft Cloudエコシステム活用

**AutoGenを選ぶべき場合:**
- 複数エージェントの協調作業
- ブレインストーミングや議論
- 役割分担による複雑なタスク処理
- 研究・教育目的

### 🛠️ 実装の特徴

**Semantic Kernelの強み:**
- プラグインシステムの柔軟性
- 企業向け機能（セキュリティ、管理）
- .NET、Python両対応
- Azure Servicesとの密結合

**AutoGenの強み:**
- エージェント間会話の自然さ
- 複雑なワークフローの表現力
- コミュニティ活動の活発さ
- オープンソースエコシステム

## 🎓 学習のまとめ

### ✅ 習得したスキル
- **高度なプラグインシステム**の設計と実装
- **エラーハンドリング**の本格的な実装
- **会話管理**とセッション制御
- **フレームワーク比較**の判断基準

### 🚀 次のステップ
- Azure Servicesとの統合実装
- カスタムコネクタの開発
- セキュリティ機能の実装
- パフォーマンス最適化手法

### 🔧 企業導入のポイント
- **セキュリティ要件**の明確化
- **スケーラビリティ**設計
- **モニタリング**と**ログ管理**
- **CI/CD**パイプライン統合